# RNA secondary structure prediction

## Import packages 
## Initialize global variables

In [2]:
import numpy as np
import sys
import math
import time
import os
from os import listdir
from os.path import isfile, join
import random
min_sepa=4 # no sharp turns

## Check RNA pair 

In [3]:
# Input:
# base1, base2: characters
def check_pair(base1, base2):
    return (base1=='A' and base2=='U') or (base1=='U' and base2=='A') or (base1=='C' and base2=='G') or (base1=='G' and base2=='C')

## Print out results

In [4]:
# align in both jupyter notebook and copy paste in text editor
# Input:
# current: RNA
def printResult(current):
    dp, pair, dura=nussinov(current)
    n=len(current)
    match=["." for i in range(n)]
    for i in pair:
        match[min(i)]="("
        match[max(i)]=")"
    match="".join(match).split()
    current=current.split()
    for i in current: print(*i, sep='\t', end='')
    print()
    for i in match: print(*i, sep='\t',end='')
    print('\nLength = ', n, ", Pairs = ", len(pair), ", Time = ", dura, " sec", sep='')
    if (n < 26):
        for line in dp:
            print(*line, sep="\t")        
    print()

## Find traceback

In [5]:
# Input:
# i, j: indices of current location
# dp: nussinov table
# result: traceback pair
# current: RNA
def traceback(i, j, dp, result, current):              
    if j > i:  
        if dp[i][j] == dp[i][j-1]:
            traceback(i, j-1, dp, result, current)
            return
        else:
            for t in range(i, j-min_sepa):
                if check_pair(current[t], current[j]):                 
                    if t==0 and dp[i][j] == dp[t+1][j-1] + 1:
                        result.append((t, j))
                        traceback(t+1, j-1, dp, result, current)
                        return
                    elif dp[i][j] == dp[i][t-1] + dp[t+1][j-1] + 1:
                        result.append((t, j))
                        traceback(i, t-1, dp, result, current)
                        traceback(t+1, j-1, dp, result, current)
                        return

## Implement Nussinov Algorithm

In [6]:
# row index i increases from top to bottom
# col index j increases from left to right
# diagonal k from bottom left to upper right
# use index convention of CSE417 slides
# Input:
# current: RNA
def nussinov(current):
    start=time.time()
    n=len(current)
    dp=np.zeros((n,n),dtype=int)
    result=[]
    
    for k in range(min_sepa, n):
        for i in range(n-k):
            j=i+k
            if i < j-min_sepa:
                max_v=dp[i][j-1]
                for t in range(i, j-min_sepa):
                    if check_pair(current[t],current[j]):
                        max_v=max(max_v, dp[i][t-1]+dp[t+1][j-1]+1)
                dp[i][j]=max_v
                dp[j][i]=dp[i][j]
    traceback(0, n-1, dp, result, current)
    dura=time.time()-start
    return dp, result, dura

## Implement Nussinov Algorithm and return separate runtimes

In [7]:
# row index i increases from top to bottom
# col index j increases from left to right
# diagonal k from bottom left to upper right
# use index convention of CSE417 slides
# Input:
# current: RNA
def nussinov_clean(current):
    start=time.time()
    n=len(current)
    dp=np.zeros((n,n))
    result=[]
    
    for k in range(min_sepa, n):
        for i in range(n-k):
            j=i+k
            if i < j-min_sepa:
                max_v=dp[i, j-1]
                for t in range(i, j-min_sepa):
                    if check_pair(current[t],current[j]):
                        max_v=max(max_v, dp[i, t-1]+dp[t+1, j-1]+1)
                dp[i,j]=max_v
                #dp[j][i]=dp[i][j]
    nusE=time.time()-start
    
    traceS=time.time()
    traceback(0, n-1, dp, result, current)
    traceE=time.time()-traceS
    
    dura=time.time()-start
    return [dura, nusE, traceE]

## Small test

In [7]:
# not align in jupyter notebook interface 
# but align after copy paste in text editor
# Input:
# current: RNA
def printResult2(current):
    dp, pair, dura=nussinov(current)
    n=len(current)
    match=["." for i in range(n)]
    for i in pair:
        match[min(i)]="("
        match[max(i)]=")"
    match="".join(match)
    print(current, '\n', match, '\nLength = ', n, ", Pairs = ", len(pair), ", Time = ", dura, " sec", sep='')
    if (n < 26):
        for line in dp:
            print(*line, sep="\t")        
    print()

In [8]:
printResult("AGCUCAUAUGGC")
printResult("GCCCACCUUCGAAAAGACUGGAUGACCAUGGGCCAUGAUU")
printResult("GCUCCAGUGGCCUAAUGGAUAUGGCUUUGGACUUCUAAUCCAAAGGUUGCGGGUUCGAGUCCCGUCUGGAGUA")

A	G	C	U	C	A	U	A	U	G	G	C
.	(	(	.	(	.	.	.	.	)	)	)
Length = 12, Pairs = 3, Time = 0.00018405914306640625 sec
0	0	0	0	0	0	1	1	1	1	2	3
0	0	0	0	0	0	0	0	0	1	2	3
0	0	0	0	0	0	0	0	0	1	2	2
0	0	0	0	0	0	0	0	0	1	1	1
0	0	0	0	0	0	0	0	0	1	1	1
0	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0
1	1	1	1	1	0	0	0	0	0	0	0
2	2	2	1	1	0	0	0	0	0	0	0
3	3	2	1	1	0	0	0	0	0	0	0

G	C	C	C	A	C	C	U	U	C	G	A	A	A	A	G	A	C	U	G	G	A	U	G	A	C	C	A	U	G	G	G	C	C	A	U	G	A	U	U
(	(	(	(	(	(	.	(	(	.	.	.	.	)	)	)	.	.	(	(	(	.	.	.	.	)	)	)	)	)	)	)	)	.	(	.	.	.	.	)
Length = 40, Pairs = 12, Time = 0.00757908821105957 sec

G	C	U	C	C	A	G	U	G	G	C	C	U	A	A	U	G	G	A	U	A	U	G	G	C	U	U	U	G	G	A	C	U	U	C	U	A	A	U	C	C	A	A	A	G	G	U	U	G	C	G	G	G	U	U	C	G	A	G	U	C	C	C	G	U	C	U	G	G	A	G	U	A
.	(	(	(	(	(	(	.	(	(	(	(	(	(	.	.	.	.	.	)	)	.	)	)	(	(	(	(	(	(	(	.	(	.	.	.	.	)	)	)	)	)	)	)	)	(	(	.	.	(	(	.	.	.	.	)	)	)	.	.	)	)	)	.	.	)	)	)	)	)	)	.	.
Length = 73, Pairs = 24, Time = 0.045770883560180664 sec



In [9]:
printResult2("AGCUCAUAUGGC")
printResult2("GCCCACCUUCGAAAAGACUGGAUGACCAUGGGCCAUGAUU")
printResult2("GCUCCAGUGGCCUAAUGGAUAUGGCUUUGGACUUCUAAUCCAAAGGUUGCGGGUUCGAGUCCCGUCUGGAGUA")

AGCUCAUAUGGC
.((.(....)))
Length = 12, Pairs = 3, Time = 0.00016880035400390625 sec
0	0	0	0	0	0	1	1	1	1	2	3
0	0	0	0	0	0	0	0	0	1	2	3
0	0	0	0	0	0	0	0	0	1	2	2
0	0	0	0	0	0	0	0	0	1	1	1
0	0	0	0	0	0	0	0	0	1	1	1
0	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0
1	0	0	0	0	0	0	0	0	0	0	0
1	1	1	1	1	0	0	0	0	0	0	0
2	2	2	1	1	0	0	0	0	0	0	0
3	3	2	1	1	0	0	0	0	0	0	0

GCCCACCUUCGAAAAGACUGGAUGACCAUGGGCCAUGAUU
((((((.((....)))..(((....)))))))).(....)
Length = 40, Pairs = 12, Time = 0.008163213729858398 sec

GCUCCAGUGGCCUAAUGGAUAUGGCUUUGGACUUCUAAUCCAAAGGUUGCGGGUUCGAGUCCCGUCUGGAGUA
.((((((.((((((.....)).))(((((((.(....))))))))((..((....)))..)))..))))))..
Length = 73, Pairs = 24, Time = 0.04955697059631348 sec



## Generate a file with random RNAs

In [8]:
# Generate random RNA
# Input: 
# length: length of RNA
def rand_rna(length):
    rna=""
    for i in range(length):
        x = random.randint(1,4)
        if x==1:
            rna += "A"
        elif x==2:
            rna += "U"
        elif x==3:
            rna += "C"
        else:
            rna += "G"
    return rna

In [9]:
# Generate a file with random RNAs
# Input:
# each: the number of RNA of each length
# lenLo: lower bound of power of 2
# lenUp: upper bound of power of 2
# out: output file name distinct suffix
# power: 1 for RNA of discrete length of 2^k, k in [lenLo, lenUp]
# power: 0 for RNA of continuous length between 2^lenLo and 2^lenUp
def gen_rna(each, lenLo, lenUp, out, power):    
    if (lenLo>lenUp):return
    outPath = './testcase/rand_rna' + str(out) + '.txt'
    try:
        os.remove(outPath)
    except OSError: pass
        
    outF = open(outPath,'a')
    n=[]
    if power:
        for i in range(lenLo, lenUp+1):
            for j in range(each):
                n.append(2**i)
    else: n=range(2**lenLo, 2**lenUp+1)
    for i in n:
            outF.write(rand_rna(i))
            outF.write("\n")
    outF.close()

## Print out runtime of Nussinov Algorithm for different RNA

In [10]:
# Input:
# out: output file name distinct suffix
def file_test(out):
    inF = './testcase/rand_rna' + str(out) + '.txt'
    outF = './testcase/test' + str(out) + '.txt'
    with open(inF) as f:  
        # create a new output file
        try:
            os.remove(outF)
        except OSError:
            pass
        
        # output report to a file
        out = open(outF,'a')
        # write a header
        out.write("length")
        out.write("\t")
        out.write("total")
        out.write("\t")
        out.write("nus")
        out.write("\t")
        out.write("trace")
        out.write("\n")
        for line in f:
            [dura, nusE, traceE] = nussinov_clean(line) 
            out.write(str(len(line)-1))
            out.write("\t")
            out.write(str(dura))
            out.write("\t")
            out.write(str(nusE))
            out.write("\t")
            out.write(str(traceE))
            out.write("\n")
        out.close()

# TEST

## RNA of length between 16 and 512

In [ ]:
out ="".join([str(2**4),'-',str(2**9)])

# Generate random RNA
gen_rna(1, 4, 9, out, 0)

# Nussinov time
file_test(out)

## RNA of length of 2^k, k in [4,11]

In [ ]:
out="4-11power"

# Generate random RNA
gen_rna(1, 4, 11, out, 1)

# Nussinov time
file_test(out)

## RNA of length of 2^k, k in [4,12]

In [ ]:
out="4-12power"

# Generate random RNA
gen_rna(1, 4, 12, out, 1)

# Nussinov time
file_test(out)

## RNA of length of 2^k, k in [4,9], 10 RNAs each length 

In [ ]:
out ="4-9powerN10"

# Generate random RNA
gen_rna(10, 4, 9, out, 1)

# Nussinov time
file_test(out)

## Test with user type-in
### Reference: RNA of length of 512 total runtime: 19.366272926330566 sec
### Reference: RNA of length of 1024 total runtime: 107.21527886390686 sec
### Reference: RNA of length of 2048 total runtime: 848.2827141284943 sec
### Reference: RNA of length of 4096 total runtime: 7077.7931871414185 sec

In [ ]:
RNA = input("Enter an RNA: [Press return to obtain result :)]")
printResult(RNA)
# Generate random RNA
out="4-9power"
gen_rna(1, 4, 9, out, 1)
# Nussinov time
file_test(out)